In [2]:
# AUTHENTICATION

import numpy as np
import hashlib
import cv2
import os
import base64
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.kdf.pbkdf2 import PBKDF2HMAC
from skimage.morphology import skeletonize

def clear_cache():
    cache_files = ["processed_sender.jpg", "processed_receiver.jpg"]
    for cache_file in cache_files:
        if os.path.exists(cache_file):
            os.remove(cache_file)

class BiometricKeyGenerator:
    def __init__(self):
        self.salt_sender = self.load_or_generate_salt("sender_salt")
        self.salt_receiver = self.load_or_generate_salt("receiver_salt")

    def load_or_generate_salt(self, filename):
        if os.path.exists(filename):
            with open(filename, "rb") as f:
                return f.read()
        else:
            salt = os.urandom(16)
            with open(filename, "wb") as f:
                f.write(salt)
            return salt

    def preprocess_fingerprint(self, image, cache_file):
        if os.path.exists(cache_file):
            return cv2.imread(cache_file, cv2.IMREAD_GRAYSCALE)

        if len(image.shape) > 2:
            image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

        image = cv2.resize(image, (500, 500))
        normalized = cv2.normalize(image, None, 0, 255, cv2.NORM_MINMAX)
        _, binary = cv2.threshold(normalized, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
        skeleton = skeletonize(binary // 255) * 255

        cv2.imwrite(cache_file, skeleton)
        return skeleton.astype(np.uint8)

    def extract_minutiae(self, skeletonized_image):
        minutiae_points = []
        rows, cols = skeletonized_image.shape

        for y in range(1, rows - 1):
            for x in range(1, cols - 1):
                if skeletonized_image[y, x] == 255:
                    neighborhood = skeletonized_image[y-1:y+2, x-1:x+2]
                    connectivity = np.count_nonzero(neighborhood) - 1

                    if connectivity == 1:
                        minutiae_points.append((x, y, 'ending'))
                    elif connectivity > 2:
                        minutiae_points.append((x, y, 'bifurcation'))

        minutiae_points = sorted(minutiae_points)
        feature_str = ','.join([f"{x}:{y}:{t}" for x, y, t in minutiae_points])

        return feature_str if feature_str else hashlib.sha256(skeletonized_image.tobytes()).hexdigest()

    def generate_key(self, feature_string, is_sender=True):
        salt = self.salt_sender if is_sender else self.salt_receiver
        feature_hash = hashlib.sha256(feature_string.encode()).digest()

        kdf = PBKDF2HMAC(
            algorithm=hashes.SHA256(),
            length=16,
            salt=salt,
            iterations=100000,
        )

        return kdf.derive(feature_hash)

    def generate_key_pair(self, sender_fingerprint, receiver_fingerprint):
        processed_sender = self.preprocess_fingerprint(sender_fingerprint, "processed_sender.jpg")
        processed_receiver = self.preprocess_fingerprint(receiver_fingerprint, "processed_receiver.jpg")

        sender_features = self.extract_minutiae(processed_sender)
        receiver_features = self.extract_minutiae(processed_receiver)

        K1 = self.generate_key(sender_features, is_sender=True)
        K2 = self.generate_key(receiver_features, is_sender=False)

        return K1, K2

    def keys_to_hex(self, K1, K2):
        return K1.hex(), K2.hex()

def main():
    try:
        clear_cache()
        sender_fingerprint_path = "s1.jpg"
        receiver_fingerprint_path = "receiver_fingerprint.jpg"

        sender_fingerprint = (cv2.imread(sender_fingerprint_path) if os.path.exists(sender_fingerprint_path)
                              else np.random.randint(0, 256, (500, 500), dtype=np.uint8))

        receiver_fingerprint = (cv2.imread(receiver_fingerprint_path) if os.path.exists(receiver_fingerprint_path)
                                else np.random.randint(0, 256, (500, 500), dtype=np.uint8))

        key_gen = BiometricKeyGenerator()
        K1, K2 = key_gen.generate_key_pair(sender_fingerprint, receiver_fingerprint)
        K1_hex, K2_hex = key_gen.keys_to_hex(K1, K2)

        return K1_hex, K2_hex

    except Exception as e:
        print(f"Error in generating keys: {str(e)}")
        return None, None

if __name__ == "__main__":
    token_1, token_2 = main()
print("token_1:"+token_1)
print("token_2:"+token_2)

token_1:ee7f118e841ea7507ce385ec47cb7c22
token_2:b4fe7ac2db33a3be9b60ae02849232b4


In [3]:
stored_token1 = "ee7f118e841ea7507ce385ec47cb7c22"
stored_token2 = "b4fe7ac2db33a3be9b60ae02849232b4"

if token_1 == stored_token1 and token_2 == stored_token2:
    print("Authentication PASS: User verified, proceeding with encryption.")
else:
    raise ValueError("Authentication FAIL: Key mismatch detected. Aborting further execution.")


Authentication PASS: User verified, proceeding with encryption.


In [4]:
# KEY GENERATION
import hashlib
import base64
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.kdf.pbkdf2 import PBKDF2HMAC

class IDBasedKeyGenerator:
    def __init__(self, sender_id, receiver_id):
        self.salt = self.derive_salt(sender_id, receiver_id)

    def derive_salt(self, sender_id, receiver_id):
        """Derive a 16-byte salt from concatenated sender and receiver IDs."""
        combined_ids = f"{sender_id}:{receiver_id}".encode()
        hash_digest = hashlib.sha256(combined_ids).digest()
        return hash_digest[:16]  # 16 bytes = 128 bits salt

    def generate_keys(self, passphrase):
        """Generate a 256-bit key and split into two 128-bit keys."""
        kdf = PBKDF2HMAC(
            algorithm=hashes.SHA256(),
            length=32,  # 256 bits total
            salt=self.salt,
            iterations=100000,
        )
        key_256 = kdf.derive(passphrase.encode())
        K1 = key_256[:16]  # First 128 bits
        K2 = key_256[16:]  # Next 128 bits
        return K1, K2

    def key_to_binary(self, key):
        return ''.join(format(byte, '08b') for byte in key)

    def key_to_hex(self, key):
        return key.hex()

    def get_salt_base64(self):
        return base64.b64encode(self.salt).decode()

# Example usage
def main():
    sender_id = "user_sender_007"
    receiver_id = "user_receiver_912"
    passphrase = str(token_1)  # One passphrase only

    print("\nGenerating 256-bit key, splitting into K1 and K2...")

    key_gen = IDBasedKeyGenerator(sender_id, receiver_id)

    K1, K2 = key_gen.generate_keys(passphrase)

    K1_binary = key_gen.key_to_binary(K1)
    K2_binary = key_gen.key_to_binary(K2)
    K1_hex = key_gen.key_to_hex(K1)
    K2_hex = key_gen.key_to_hex(K2)

    print("\n===== GENERATED ENCRYPTION KEYS =====")
    print("K1:")
    print(f"Binary (128-bit): {K1_binary}")
    print(f"Hex (for reference): {K1_hex}")

    print("\nK2:")
    print(f"Binary (128-bit): {K2_binary}")
    print(f"Hex (for reference): {K2_hex}")

    print("\nDerived Salt (base64):")
    print(key_gen.get_salt_base64())

    print("\nKeys generated successfully!")

    return K1_binary, K2_binary

if __name__ == "__main__":
    K1, K2 = main()



Generating 256-bit key, splitting into K1 and K2...

===== GENERATED ENCRYPTION KEYS =====
K1:
Binary (128-bit): 00001001101110100011111111011110100110100101000001010010111000111011110001010101000010011000101000011101100010101101101100000101
Hex (for reference): 09ba3fde9a5052e3bc55098a1d8adb05

K2:
Binary (128-bit): 00101001101111100101100101110001101111111111110111000110001001101011101000011010110011110111100010111100011100100100101111000010
Hex (for reference): 29be5971bffdc626ba1acf78bc724bc2

Derived Salt (base64):
yhTTBy6f8YnF2AFud4RGPA==

Keys generated successfully!


In [5]:
realoriginal_K1 = K1
realoriginal_K2 = K2

In [6]:
print(realoriginal_K1)

00001001101110100011111111011110100110100101000001010010111000111011110001010101000010011000101000011101100010101101101100000101


In [7]:
import time
start_time = time.time()

In [8]:
from PIL import Image

In [9]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
from math import lcm
import pickle

def resize_image_to_universal_blocks(image_path, block_sizes):
    """
    Resizes an image so that it can be divided into any of the given block sizes without remainder pixels.

    Args:
        image_path (str): Path to the input image.
        block_sizes (list of int): List of block sizes (e.g., [16, 32, 64]).

    Returns:
        np.ndarray: The resized image.
    """
    # Load the image
    image = cv2.imread(image_path)
    if image is None:
        raise ValueError("Invalid image path")
    
    original_height, original_width = image.shape[:2]
    print(f"Original Image Dimensions: {original_width}x{original_height}")
    
    # Compute the LCM of the block sizes
    target_block_size = lcm(*block_sizes)
    print(f"Target Block Size (LCM of {block_sizes}): {target_block_size}")
    
    # Calculate the new dimensions as the nearest multiples of the LCM
    new_width = (original_width // target_block_size) * target_block_size
    new_height = (original_height // target_block_size) * target_block_size
    
    # Resize the image
    resized_image = cv2.resize(image, (new_width, new_height), interpolation=cv2.INTER_AREA)
    print(f"Resized Image Dimensions: {new_width}x{new_height}")
    
    cv2.imwrite(save_path, resized_image)
    return resized_image
    
image_path = "The-Taj-Mahal-Agra-India.jpg"
save_path = "original_image.png"

block_sizes = [16, 32, 64]
resized_image = resize_image_to_universal_blocks(image_path, block_sizes)

# def predictive_encode(image):
#     """
#     Apply predictive coding on an RGB image.
#     Each pixel is predicted from the left neighbor.
#     """
#     image_array = np.array(image, dtype=np.int16)  # Convert to int16 to handle negative residuals
#     height, width, channels = image_array.shape
#     residual = np.zeros_like(image_array)

#     for y in range(height):
#         for x in range(width):
#             for c in range(channels):
#                 if x == 0:
#                     predicted = 0
#                 else:
#                     predicted = image_array[y, x - 1, c]
#                 residual[y, x, c] = image_array[y, x, c] - predicted

#     return residual

# input_image = Image.open("original_image.png").convert("RGB")

# # Encode
# residual_image = predictive_encode(input_image)

# # Optionally save residual as image for inspection (after normalization)
# residual_vis = np.clip(residual_image + 128, 0, 255).astype(np.uint8)  # Shift for visibility
# Image.fromarray(residual_vis).save("original_image_1.png")
# with open("residual.bin", "wb") as f:
#     pickle.dump(residual_image, f)

Original Image Dimensions: 274x184
Target Block Size (LCM of [16, 32, 64]): 64
Resized Image Dimensions: 256x128


In [10]:
from PIL import Image
import os
import numpy as np
import random
import secrets

def clear_folder(folder_path):
    """Deletes all files inside a folder."""
    if os.path.exists(folder_path):
        for file in os.listdir(folder_path):
            os.remove(os.path.join(folder_path, file))
    os.makedirs(folder_path, exist_ok=True)

block_sizes = [16, 32, 64]
image_path = 'original_image.png'  # Initial image

for block_size in block_sizes:
    print(f"Processing with block size {block_size}...")
        
    # Remove previous blocks before creating new ones
    clear_folder("image_blocks")  
    clear_folder("transformed_image_blocks")
    
    # Load the image (use the last scrambled image in later iterations)
    img = Image.open(image_path)
    width, height = img.size
    
    # Calculate number of full blocks in each dimension
    num_blocks_x = width // block_size
    num_blocks_y = height // block_size
    
    # Split the image into blocks and save them
    block_count = 1
    for y in range(0, num_blocks_y):
        for x in range(0, num_blocks_x):
            # Crop each block
            left = x * block_size
            top = y * block_size
            right = left + block_size
            bottom = top + block_size
            block = img.crop((left, top, right, bottom))
            
            # Save the block with a unique filename
            block_filename = os.path.join("image_blocks", f"block_{block_count:06d}.png")
            block.save(block_filename)
            block_count += 1
    
    # Define the encryption matrix
    matrix = {
        (0, 0): 'L', (0, 1): 'R', (0, 2): 'U', (0, 3): 'D',
        (1, 0): 'F', (1, 1): "L'", (1, 2): "R'", (1, 3): "U'",
        (2, 0): "D'", (2, 1): "F'", (2, 2): 'L2', (2, 3): 'R2',
        (3, 0): 'U2', (3, 1): 'D2', (3, 2): 'F2', (3, 3): 'L'
    }
    
    # Store scrambling sequences for each row
    scrambling_sequences = []
    
    for i in range(num_blocks_y):  # Iterate over each row
        operations = []
        for j in range(0, 128, 4):
            part_1 = int(K1[j:j+4], 2) % 16
            temp = K2[j:j+4]
            row = int(temp[:2], 2)
            col = int(temp[2:], 2)
            part_2 = matrix[(row, col)]
            operations.append(str(part_1 + 1) + str(part_2))
        scrambling_sequences.append(operations)
        
        # Store original values before updating
        original_K1 = K1
        original_K2 = K2
    
        # Key update logic for next row
        K1 = bin((int(original_K1, 2) << 1) & ((1 << 128) - 1))[2:].zfill(128)
        K2 = bin((int(original_K2, 2) << 1) & ((1 << 128) - 1))[2:].zfill(128)
    
        K1 = bin(int(K1, 2) ^ int(original_K2, 2))[2:].zfill(128)
        K2 = bin(int(K2, 2) ^ int(original_K1, 2))[2:].zfill(128)
        
        # Bit flipping
        K1 = list(K1)
        K2 = list(K2)
        K1[i % 128] = '1' if K1[i % 128] == '0' else '0'
        K2[i % 128] = '1' if K2[i % 128] == '0' else '0'
        K1 = ''.join(K1)
        K2 = ''.join(K2)
    
    # Print stored scrambling sequences
    for idx, seq in enumerate(scrambling_sequences):
        print(f"Row {idx + 1} scrambling sequence: {seq}")
    
    import os
    import numpy as np
    from PIL import Image
    import re
    
    # Function to rotate elements in an array
    def rotate(arr, steps):
        return np.roll(arr, steps)
    
    # Function to rotate a matrix
    def rotate_matrix(mat, clockwise=True, times=1):
        for _ in range(times):
            mat = np.rot90(mat, -1 if clockwise else 1)
        return mat
    
    # Function to apply operations to the matrix
    def apply_operations(matrix, operations):
        for op in operations:
            match = re.match(r"(\d+)([RLDUF])(['\d]*)", op)
            if match:
                num_part = int(match.group(1))  # Extract numeric part
                letter = match.group(2)  # Extract operation type (R, L, U, D, F)
                extra_part = match.group(3)  # Extract any extra part (' or additional digits)
                
                # print(f"Applying operation: {op}")
                
                # Default shift is 1 (left shift for rows, downward shift for columns)
                steps = 1  
                counter_rotation = "'" in extra_part  # Check for counter-rotation
    
                # If numeric value exists in extra_part, update steps
                if extra_part and extra_part.replace("'", "").isdigit():
                    steps = int(extra_part.replace("'", ""))  # Use the numeric value in extra_part
                
                if letter == 'R':  # Rotate a column (R)
                    col_idx = matrix.shape[1] - num_part  # Convert to column index from the right
                    if counter_rotation:
                        matrix[:, col_idx] = rotate(matrix[:, col_idx], steps)  # Rotate upwards (R')
                    else:
                        matrix[:, col_idx] = rotate(matrix[:, col_idx], -steps)  # Default downward shift (R)
    
                elif letter == 'L':  # Rotate a column (L)
                    col_idx = num_part - 1  # Convert to column index from the left
                    if counter_rotation:
                        matrix[:, col_idx] = rotate(matrix[:, col_idx], steps)  # Rotate upwards (L')
                    else:
                        matrix[:, col_idx] = rotate(matrix[:, col_idx], -steps)  # Default downward shift (L)
    
                elif letter == 'D':  # Rotate a row (D)
                    row_idx = matrix.shape[0] - num_part  # Convert to row index from the bottom
                    if counter_rotation:
                        matrix[row_idx, :] = rotate(matrix[row_idx, :], -steps)  # Left shift (D')
                    else:
                        matrix[row_idx, :] = rotate(matrix[row_idx, :], steps)  # Default right shift (D)
    
                elif letter == 'U':  # Rotate a row (U)
                    row_idx = num_part - 1  # Convert to row index from the top
                    if counter_rotation:
                        matrix[row_idx, :] = rotate(matrix[row_idx, :], steps)  # Right shift (U')
                    else:
                        matrix[row_idx, :] = rotate(matrix[row_idx, :], -steps)  # Default left shift (U)
    
                elif letter == 'F':  # Rotate the full matrix (F)
                    if counter_rotation:
                        matrix = rotate_matrix(matrix, clockwise=False, times=steps)  # Counter-clockwise (F')
                    else:
                        matrix = rotate_matrix(matrix, clockwise=True, times=steps)  # Clockwise (F)
        
        return matrix
    
    # Function to process an image block
    def process_block(block_path, operations, output_folder):
        image = Image.open(block_path)
        block_no = int(os.path.basename(block_path).split('_')[1].split('.')[0])
        r, g, b = np.array(image).transpose(2, 0, 1)
    
        # Print the scrambling sequence for the current block
        print(f"Scrambling sequence for block {block_no}: {operations}")
        
        r_transformed = apply_operations(r, operations)
        g_transformed = apply_operations(g, operations)
        b_transformed = apply_operations(b, operations)
        
        transformed_image = np.stack([r_transformed, g_transformed, b_transformed], axis=2)
        transformed_image = Image.fromarray(transformed_image.astype(np.uint8))
        
        os.makedirs(output_folder, exist_ok=True)
        output_path = os.path.join(output_folder, f"transformed_block_{block_no:06d}.png")
        transformed_image.save(output_path)
        print(f"Saved {output_path}")
    
    # Main function to process all blocks
    def main():
        input_folder = "image_blocks"
        output_folder = "transformed_image_blocks"
        
        # Load the scrambling sequence matrix
        global scrambling_sequences  # Ensure we use the previously generated matrix
        
        block_files = sorted(os.listdir(input_folder))
        num_blocks_x = len(block_files) // len(scrambling_sequences)  # Calculate blocks per row
        
        for row_idx in range(len(scrambling_sequences)):
            row_operations = scrambling_sequences[row_idx]  # Use the corresponding scrambling sequence
            
            for col_idx in range(num_blocks_x):
                block_no = row_idx * num_blocks_x + col_idx + 1
                block_path = os.path.join(input_folder, f"block_{block_no:06d}.png")
                
                if os.path.exists(block_path):
                    process_block(block_path, row_operations, output_folder)
                else:
                    print(f"Block {block_no} not found, skipping...")
    
    if __name__ == "__main__":
        main()
    
    import os
    from PIL import Image
    import numpy as np
    
    input_folder = "transformed_image_blocks"  # Folder containing transformed blocks
    output_image_path = "final_scrambled_image.png"
    
    # Get block filenames and sort them
    block_files = sorted(f for f in os.listdir(input_folder) if f.endswith(('.png', '.jpg', '.jpeg')))
    
    # Calculate number of full blocks in each dimension
    # num_blocks = len(block_files)
    num_blocks_x = width // block_size
    num_blocks_y = height // block_size
    
    # Create a blank canvas for the final image
    final_image = Image.new("RGB", (num_blocks_x * block_size, num_blocks_y * block_size))
    
    # Merge blocks back into the final image
    block_index = 0
    for y in range(num_blocks_y):
        for x in range(num_blocks_x):
            block_path = os.path.join(input_folder, block_files[block_index])
            block = Image.open(block_path)
            final_image.paste(block, (x * block_size, y * block_size))
            block_index += 1
    
    # Save the reconstructed image
    final_image.save(output_image_path)
    
    # Update image path for the next iteration
    image_path = "final_scrambled_image.png"

    K1=realoriginal_K1
    K2=realoriginal_K2
    
    print(f"Final scrambled image saved as {output_image_path}")
    
print("Process completed for all block sizes.")    
# Load the image
image_path = 'final_scrambled_image.png'
img = Image.open(image_path)

Processing with block size 16...
Row 1 scrambling sequence: ['1U', "10F'", '12R2', '11F2', "4L'", "16F'", "14U'", '15R', '10R2', '11L', '6L', '1D2', '6U2', "3R'", '15U', "4R'", '12R2', '13L2', '6R', '6L2', '1U2', '10L', "9U'", "11D'", '2R2', '14U2', "9U'", '11U', '14F', '12R2', '1U2', '6U']
Row 2 scrambling sequence: ['12D2', '11L2', '13U2', "11R'", "3D'", '7D2', '13D', '13D2', '9F2', "12L'", '6L2', '14R2', '7D2', '4F2', '15L2', '2F2', '13U2', "3D'", "12R'", '1L', "14F'", "13U'", "7U'", '13R2', "9R'", "8L'", "7R'", '8F2', '16F', '14U2', "13D'", '9R']
Row 3 scrambling sequence: ['15F', '16L', '6F', "3U'", '13D', '1U2', '11R2', "5U'", '16F', '4L', '2L', '2L2', '2D2', '10F2', '7R2', '14U2', "5L'", '14U', "1U'", '2R', '3L', '16U', "11F'", '3L2', '7F', '12D2', '11R2', '2R2', "12R'", '8F', '2U2', '2L2']
Row 4 scrambling sequence: ["12L'", '2R', '15D2', '3U2', '12R2', "14F'", '16U2', '15L2', "11U'", '7D', '3L', '9F', '15L2', '13F', '7R', "7L'", '13F2', "9F'", '8F2', '4U', '11U2', '14L2', "14F

In [11]:
import numpy as np
from PIL import Image

# Load the image (replace the path with your image path)
img = Image.open('final_scrambled_image.png')
numpydata = np.array(img)

# Extract RGB channels
red_channel = numpydata[:, :, 0]
green_channel = numpydata[:, :, 1]
blue_channel = numpydata[:, :, 2]

# Create a dictionary to store the bit-planes for each channel
bit_planes = {
    'Red': {},
    'Green': {},
    'Blue': {}
}

# Function to generate bit-planes for a given channel
def generate_bit_planes(channel, channel_name):
    for bit in range(8):  # Iterate from LSB to MSB (bit 0 to bit 7)
        # Right shift and apply bitwise AND to extract the specific bit-plane
        bit_plane = (channel >> bit) & 1
        bit_planes[channel_name][bit] = bit_plane
        print(f"{channel_name} Channel - Bit-plane {bit}:")
        print(bit_plane)  # This will print the whole bit-plane matrix
        print()

# Generate bit-planes for each channel
generate_bit_planes(red_channel, 'Red')
generate_bit_planes(green_channel, 'Green')
generate_bit_planes(blue_channel, 'Blue')


# Create an array to store all bit-planes (shape: (24, height, width))
height, width = red_channel.shape
bitplane_array = np.zeros((24, height, width), dtype=int)

# Populate the bitplane_array with the respective bit-planes
for bit in range(8):
    # Red channel bit-planes (0–7)
    bitplane_array[bit] = bit_planes['Red'][bit]
    
    # Green channel bit-planes (8–15)
    bitplane_array[bit + 8] = bit_planes['Green'][bit]
    
    # Blue channel bit-planes (16–23)
    bitplane_array[bit + 16] = bit_planes['Blue'][bit]

# The bitplane_array now holds the 24 bit-planes for the RGB channels in LSB to MSB order
print(bitplane_array.shape)  # Should print (24, height, width)


Red Channel - Bit-plane 0:
[[1 1 0 ... 1 1 1]
 [1 1 0 ... 1 1 1]
 [1 0 0 ... 1 1 1]
 ...
 [1 0 0 ... 0 1 1]
 [1 1 1 ... 1 0 1]
 [0 0 0 ... 1 1 1]]

Red Channel - Bit-plane 1:
[[1 1 0 ... 0 1 0]
 [0 1 0 ... 1 0 1]
 [1 1 1 ... 1 1 1]
 ...
 [1 0 0 ... 0 1 1]
 [1 1 1 ... 0 0 1]
 [0 1 1 ... 0 0 1]]

Red Channel - Bit-plane 2:
[[1 1 1 ... 0 0 1]
 [1 1 0 ... 1 1 1]
 [0 1 1 ... 1 1 1]
 ...
 [0 0 1 ... 0 1 0]
 [0 1 0 ... 0 1 1]
 [0 1 0 ... 1 1 0]]

Red Channel - Bit-plane 3:
[[1 0 0 ... 1 0 1]
 [0 1 1 ... 1 1 1]
 [0 1 1 ... 1 1 1]
 ...
 [0 0 1 ... 1 0 0]
 [0 0 1 ... 1 0 0]
 [0 0 0 ... 1 0 0]]

Red Channel - Bit-plane 4:
[[0 0 1 ... 0 0 0]
 [0 0 0 ... 1 1 1]
 [1 0 1 ... 1 1 1]
 ...
 [1 1 0 ... 0 0 0]
 [1 0 1 ... 1 0 1]
 [0 1 1 ... 0 1 0]]

Red Channel - Bit-plane 5:
[[0 0 1 ... 0 0 0]
 [1 0 1 ... 1 1 1]
 [1 0 0 ... 1 1 1]
 ...
 [1 0 0 ... 0 0 1]
 [0 1 1 ... 1 1 1]
 [0 1 0 ... 0 0 0]]

Red Channel - Bit-plane 6:
[[1 1 0 ... 1 1 1]
 [1 1 0 ... 1 1 1]
 [0 1 0 ... 1 1 1]
 ...
 [0 1 1 ... 1 1 1]
 [1 

In [12]:
print(realoriginal_K1)

00001001101110100011111111011110100110100101000001010010111000111011110001010101000010011000101000011101100010101101101100000101


In [13]:
K1=realoriginal_K1
K2=realoriginal_K2

In [14]:
def discard_every_4th_bit(key):
    """
    Discards every 4th bit from a 128-bit key, reducing it to 96 bits.
    
    Args:
        key (str): A 128-bit key represented as a binary string.
    
    Returns:
        str: A 96-bit key represented as a binary string.
    """
    return ''.join([bit for i, bit in enumerate(key, start=1) if i % 4 != 0])

K1_96 = discard_every_4th_bit(K1)
K2_96 = discard_every_4th_bit(K2)

import numpy as np

def generate_8bit_sequences(K1_96, K2_96):
    """Generates 24 8-bit sequences using two 96-bit keys."""
    sequences = []
    for i in range(0, 96, 4):
        seq = K1_96[i:i+4] + K2_96[i:i+4]
        sequences.append(seq)
    return sequences[:24]  # Ensure we have exactly 24 sequences

def extract_rotation_and_offset(sequence):
    """Extracts the rotation count and offset value from an 8-bit sequence."""
    rotation_count = int(sequence[0] + sequence[-1], 2)  # First and last bit for rotation count
    offset_value = int(sequence[1:7], 2)  # Middle 6 bits for offset
    return rotation_count, offset_value

def rotate_frame(bitplane, frame_number, rotation_count, offset_value):
    """
    Rotates the specified frame within a bitplane.
    
    Args:
        bitplane (numpy array): The bitplane matrix.
        frame_number (int): Index of the frame (0 = outermost).
        rotation_count (int): Number of positions to rotate.
        offset_value (int): Additional offset for rotation.
    
    Returns:
        numpy array: Bitplane with rotated frame.
    """
    H, W = bitplane.shape
    top, bottom = frame_number, H - frame_number - 1
    left, right = frame_number, W - frame_number - 1
    
    if top >= bottom or left >= right:
        return bitplane  # No more frames to rotate

    frame_pixels = []

    # Extract frame pixels in order (Top Row, Right Column, Bottom Row, Left Column)
    frame_pixels.extend(bitplane[top, left:right+1])   # Top row
    frame_pixels.extend(bitplane[top+1:bottom, right]) # Right column
    frame_pixels.extend(bitplane[bottom, left:right+1][::-1]) # Bottom row (reversed)
    frame_pixels.extend(bitplane[top+1:bottom, left][::-1])   # Left column (reversed)

    # Calculate actual shift
    shift_amount = (rotation_count * max(H, W)) + offset_value
    shift_amount %= len(frame_pixels)  # Avoid excessive rotation

    # Perform rotation
    frame_pixels = frame_pixels[-shift_amount:] + frame_pixels[:-shift_amount]

    # Put rotated pixels back into bitplane
    idx = 0
    bitplane[top, left:right+1] = frame_pixels[idx:idx + (right - left + 1)]
    idx += (right - left + 1)
    bitplane[top+1:bottom, right] = frame_pixels[idx:idx + (bottom - top - 1)]
    idx += (bottom - top - 1)
    bitplane[bottom, left:right+1] = frame_pixels[idx:idx + (right - left + 1)][::-1]
    idx += (right - left + 1)
    bitplane[top+1:bottom, left] = frame_pixels[idx:idx + (bottom - top - 1)][::-1]

    return bitplane

# Generate 8-bit sequences
bitplane_sequences = generate_8bit_sequences(K1_96, K2_96)

# Loop over bitplanes
for bitplane_index, bitplane in enumerate(bitplane_array):
    # Extract rotation and offset for this bitplane
    rotation_count, offset_value = extract_rotation_and_offset(bitplane_sequences[bitplane_index])

    print(f"Bitplane {bitplane_index + 1}: Sequence = {bitplane_sequences[bitplane_index]}, Rotations = {rotation_count}, Offset = {offset_value}")

    # Compute number of frames
    H, W = bitplane.shape
    num_frames = min(H, W) // 2  # Formula for number of frames

    # Apply rotation to each frame
    for frame in range(num_frames):
        bitplane_array[bitplane_index] = rotate_frame(bitplane_array[bitplane_index], frame, rotation_count, offset_value)

print("Frame rotation complete.")

Bitplane 1: Sequence = 00010011, Rotations = 1, Offset = 9
Bitplane 2: Sequence = 00100010, Rotations = 0, Offset = 17
Bitplane 3: Sequence = 11011111, Rotations = 3, Offset = 47
Bitplane 4: Sequence = 00110101, Rotations = 1, Offset = 26
Bitplane 5: Sequence = 11110001, Rotations = 3, Offset = 56
Bitplane 6: Sequence = 01111000, Rotations = 0, Offset = 60
Bitplane 7: Sequence = 10011011, Rotations = 3, Offset = 13
Bitplane 8: Sequence = 01011111, Rotations = 1, Offset = 47
Bitplane 9: Sequence = 00001110, Rotations = 0, Offset = 7
Bitplane 10: Sequence = 01001100, Rotations = 0, Offset = 38
Bitplane 11: Sequence = 01111100, Rotations = 0, Offset = 62
Bitplane 12: Sequence = 10011011, Rotations = 3, Offset = 13
Bitplane 13: Sequence = 10111011, Rotations = 3, Offset = 29
Bitplane 14: Sequence = 10010100, Rotations = 2, Offset = 10
Bitplane 15: Sequence = 00100101, Rotations = 1, Offset = 18
Bitplane 16: Sequence = 00011101, Rotations = 1, Offset = 14
Bitplane 17: Sequence = 00101101, R

In [15]:
import numpy as np
import cv2

def shuffle_bitplanes(bitplane_array):
    """
    Shuffles the 24 bitplanes as per the given order.
    
    Args:
        bitplane_array (numpy array): The original bitplane array (24, H, W).
    
    Returns:
        numpy array: Shuffled bitplane array (24, H, W).
    """
    shuffled_bitplanes = np.zeros_like(bitplane_array)

    # Red channel shuffle (0-7 → 7-0)
    for i in range(8):
        shuffled_bitplanes[i] = bitplane_array[7 - i]

    # Green channel shuffle (8-15 → 15-8)
    for i in range(8, 16):
        shuffled_bitplanes[i] = bitplane_array[23 - (i - 8)]

    # Blue channel shuffle (16-23 → 23-16) 
    for i in range(16, 24):
        shuffled_bitplanes[i] = bitplane_array[31 - i]  # Corrected indexing

    return shuffled_bitplanes

def integrate_bitplanes(shuffled_bitplanes):
    """
    Integrates shuffled bitplanes to reconstruct the encrypted image.
    
    Args:
        shuffled_bitplanes (numpy array): Shuffled bitplane array (24, H, W).
    
    Returns:
        numpy array: Reconstructed image (H, W, 3).
    """
    H, W = shuffled_bitplanes.shape[1], shuffled_bitplanes.shape[2]

    # Initialize color channels as uint8
    R = np.zeros((H, W), dtype=np.uint8)
    G = np.zeros((H, W), dtype=np.uint8)
    B = np.zeros((H, W), dtype=np.uint8)

    # Reconstruct color channels
    for i in range(8):  
        R |= (shuffled_bitplanes[i].astype(np.uint8) << i)  # Red channel ✅ Fix

    for i in range(8, 16):  
        G |= (shuffled_bitplanes[i].astype(np.uint8) << (i - 8))  # Green channel ✅ Fix

    for i in range(16, 24):  
        B |= (shuffled_bitplanes[i].astype(np.uint8) << (i - 16))  # Blue channel ✅ Fix

    # Merge color channels into final encrypted image
    encrypted_image = cv2.merge([B, G, R])  # OpenCV uses BGR format

    return encrypted_image


# Shuffle bitplanes
shuffled_bitplanes = shuffle_bitplanes(bitplane_array)

# Integrate shuffled bitplanes into final image
encrypted_image = integrate_bitplanes(shuffled_bitplanes)

# Ensure it's uint8 before saving
encrypted_image = encrypted_image.astype(np.uint8)

# Save the final encrypted image
cv2.imwrite("encrypted_image.png", encrypted_image)

print("Encrypted image saved successfully!")

Encrypted image saved successfully!


In [16]:
end_time = time.time()
print(f"Total execution time: {end_time - start_time:.5f} seconds")

Total execution time: 0.61490 seconds


In [17]:
import pickle
import os

def save_encryption_state(K1, K2, scrambling_sequences, bitplane_array, output_path="encryption_state.pkl"):
    """
    Saves the encryption state, including keys, scrambling sequences, and bitplane array.
    
    Args:
        K1 (str): The first 128-bit key in binary format.
        K2 (str): The second 128-bit key in binary format.
        scrambling_sequences (list): List of scrambling sequences used for block transformations.
        bitplane_array (numpy array): The final bitplane array after rotations.
        output_path (str): Path to save the encryption state file.
    """
    # Create a dictionary to store all components
    encryption_state = {
        'K1': K1,
        'K2': K2,
        'scrambling_sequences': scrambling_sequences,
        'bitplane_array': bitplane_array
    }
    
    # Save to file using pickle
    try:
        with open(output_path, 'wb') as f:
            pickle.dump(encryption_state, f)
        print(f"Encryption state saved successfully to {output_path}")
    except Exception as e:
        print(f"Error saving encryption state: {str(e)}")

def load_encryption_state(input_path="encryption_state.pkl"):
    """
    Loads the encryption state from a file.
    
    Args:
        input_path (str): Path to the saved encryption state file.
    
    Returns:
        dict: Dictionary containing the encryption state components.
    """
    if not os.path.exists(input_path):
        raise FileNotFoundError(f"Encryption state file not found at {input_path}")
    
    try:
        with open(input_path, 'rb') as f:
            encryption_state = pickle.load(f)
        print(f"Encryption state loaded successfully from {input_path}")
        return encryption_state
    except Exception as e:
        print(f"Error loading encryption state: {str(e)}")
        return None

# Example usage (to be added in the notebook after the encryption process)
if __name__ == "__main__":
    # Assuming these variables are available from the notebook
    # K1, K2: The binary keys (realoriginal_K1, realoriginal_K2)
    # scrambling_sequences: The list of scrambling sequences used in block transformations
    # bitplane_array: The final bitplane array after frame rotations
    
    save_encryption_state(
        K1=realoriginal_K1,
        K2=realoriginal_K2,
        scrambling_sequences=scrambling_sequences,
        bitplane_array=bitplane_array,
        output_path="encryption_state.pkl"
    )
    
    # Optionally, test loading the state
    loaded_state = load_encryption_state("encryption_state.pkl")
    if loaded_state:
        print("Loaded state keys:", loaded_state.keys())

Encryption state saved successfully to encryption_state.pkl
Encryption state loaded successfully from encryption_state.pkl
Loaded state keys: dict_keys(['K1', 'K2', 'scrambling_sequences', 'bitplane_array'])
